In [ ]:
from nbdev import *

In [ ]:
#default_exp layers

Cells will be exported to model_constructor.layers,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Layers

> Basic layers for constructor.

In [ ]:
#hide
# from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#export
import torch.nn as nn
import torch
from torch.nn.utils import spectral_norm
from collections import OrderedDict

# Flatten

In [ ]:
#export
class Flatten(nn.Module):
    '''flat x to vector'''
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.size(0), -1)

# Noop

In [ ]:
#export
def noop(x):
    return x


class Noop(nn.Module):
    '''Dummy module'''
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x

# ConvLayer

In [ ]:
#export
act_fn = nn.ReLU(inplace=True)


class ConvLayer(nn.Sequential):
    """Basic conv layers block"""
    Conv2d = nn.Conv2d

    def __init__(self, ni, nf, ks=3, stride=1,
                 act=True, act_fn=act_fn,
                 bn_layer=True, bn_1st=True, zero_bn=False,
                 padding=None, bias=False, groups=1, **kwargs):

        if padding is None:
            padding = ks // 2
        layers = [('conv', self.Conv2d(ni, nf, ks, stride=stride, padding=padding, bias=bias, groups=groups))]
        act_bn = [('act_fn', act_fn)] if act else []
        if bn_layer:
            bn = nn.BatchNorm2d(nf)
            nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
            act_bn += [('bn', bn)]
        if bn_1st:
            act_bn.reverse()
        layers += act_bn
        super().__init__(OrderedDict(layers))

In [ ]:
conv_layer = ConvLayer(32, 64)
conv_layer

ConvLayer(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
conv_layer = ConvLayer(32, 64, act=False)
conv_layer

ConvLayer(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
conv_layer = ConvLayer(32, 64, bn_layer=False)
conv_layer

ConvLayer(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
conv_layer = ConvLayer(32, 64, bn_1st=True)
conv_layer

ConvLayer(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
conv_layer = ConvLayer(32, 64, bn_1st=True, act_fn=nn.LeakyReLU())
conv_layer

ConvLayer(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): LeakyReLU(negative_slope=0.01)
)

In [ ]:
#hide
bs = 8
xb = torch.randn(bs, 32, 32, 32)
y = conv_layer(xb)
y.shape
assert y.shape == torch.Size([bs, 64, 32, 32])

# SimpleSelfAttention

In [ ]:
#export
# SA module from mxresnet at fastai. todo - add persons!!!
# Unmodified from https://github.com/fastai/fastai/blob/5c51f9eabf76853a89a9bc5741804d2ed4407e49/fastai/layers.py
def conv1d(ni: int, no: int, ks: int = 1, stride: int = 1, padding: int = 0, bias: bool = False):
    "Create and initialize a `nn.Conv1d` layer with spectral normalization."
    conv = nn.Conv1d(ni, no, ks, stride=stride, padding=padding, bias=bias)
    nn.init.kaiming_normal_(conv.weight)
    if bias:
        conv.bias.data.zero_()
    return spectral_norm(conv)


# Adapted from SelfAttention layer at
# https://github.com/fastai/fastai/blob/5c51f9eabf76853a89a9bc5741804d2ed4407e49/fastai/layers.py
# Inspired by https://arxiv.org/pdf/1805.08318.pdf
class SimpleSelfAttention(nn.Module):
    def __init__(self, n_in: int, ks=1, sym=False):  # , n_out:int):
        super().__init__()
        self.conv = conv1d(n_in, n_in, ks, padding=ks // 2, bias=False)
        self.gamma = nn.Parameter(torch.tensor([0.]))
        self.sym = sym
        self.n_in = n_in

    def forward(self, x):
        if self.sym:
            # symmetry hack by https://github.com/mgrankin
            c = self.conv.weight.view(self.n_in, self.n_in)
            c = (c + c.t()) / 2
            self.conv.weight = c.view(self.n_in, self.n_in, 1)
        size = x.size()
        x = x.view(*size[:2], -1)   # (C,N)
        # changed the order of mutiplication to avoid O(N^2) complexity
        # (x*xT)*(W*x) instead of (x*(xT*(W*x)))
        convx = self.conv(x)   # (C,C) * (C,N) = (C,N)   => O(NC^2)
        xxT = torch.bmm(x, x.permute(0, 2, 1).contiguous())   # (C,N) * (N,C) = (C,C)   => O(NC^2)
        o = torch.bmm(xxT, convx)   # (C,C) * (C,N) = (C,N)   => O(NC^2)
        o = self.gamma * o + x
        return o.view(*size).contiguous()

# SE Block

In [ ]:
#export
class SEBlock(nn.Module):
    "se block"
    se_layer = nn.Linear
    act_fn = nn.ReLU(inplace=True)

    def __init__(self, c, r=16):
        super().__init__()
        ch = c // r
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            OrderedDict([('fc_reduce', self.se_layer(c, ch, bias=False)),
                         ('se_act', self.act_fn),
                         ('fc_expand', self.se_layer(ch, c, bias=False)),
                         ('sigmoid', nn.Sigmoid())
                         ]))

    def forward(self, x):
        bs, c, _, _ = x.shape
        y = self.squeeze(x).view(bs, c)
        y = self.excitation(y).view(bs, c, 1, 1)
        return x * y.expand_as(x)

In [ ]:
#hide
#         return torch.mul(x, y.expand_as(x))# 

In [ ]:
se_block = SEBlock(128)
se_block

SEBlock(
  (squeeze): AdaptiveAvgPool2d(output_size=1)
  (excitation): Sequential(
    (fc_reduce): Linear(in_features=128, out_features=8, bias=False)
    (se_act): ReLU(inplace=True)
    (fc_expand): Linear(in_features=8, out_features=128, bias=False)
    (sigmoid): Sigmoid()
  )
)

In [ ]:
#hide
bs = 8
xb = torch.randn(bs, 128, 32, 32)
y = se_block(xb)
print(y.shape)
assert y.shape == torch.Size([bs, 128, 32, 32]), f"size"

torch.Size([8, 128, 32, 32])


# SEBlockConv

In [ ]:
#export
class SEBlockConv(nn.Module):
    "se block with conv on excitation"
    se_layer = nn.Conv2d
    act_fn = nn.ReLU(inplace=True)

    def __init__(self, c, r=16):
        super().__init__()
#         c_in = math.ceil(c//r/8)*8
        c_in = c // r
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            OrderedDict([
                ('conv_reduce', self.se_layer(c, c_in, 1, bias=False)),
                ('se_act', self.act_fn),
                ('conv_expand', self.se_layer(c_in, c, 1, bias=False)),
                ('sigmoid', nn.Sigmoid())
            ]))

    def forward(self, x):
        y = self.squeeze(x)
        y = self.excitation(y)
        return x * y.expand_as(x)

In [ ]:
se_block = SEBlockConv(128)
se_block

SEBlockConv(
  (squeeze): AdaptiveAvgPool2d(output_size=1)
  (excitation): Sequential(
    (conv_reduce): Conv2d(128, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (se_act): ReLU(inplace=True)
    (conv_expand): Conv2d(8, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (sigmoid): Sigmoid()
  )
)

In [ ]:
#hide
bs = 8
xb = torch.randn(bs, 128, 32, 32)
y = se_block(xb)
print(y.shape)
assert y.shape == torch.Size([bs, 128, 32, 32]), f"size"

torch.Size([8, 128, 32, 32])


# end
model_constructor
by ayasyrev

In [ ]:
#hide
from nbdev.export import *
notebook2script()

UsageError: Line magic function `%nbdev_hide` not found.
